Several doctests refer to these functions:

In [1]:
from operator import add,mul
identity = lambda x:x
square = lambda x : x*x
increment = lambda x : x+1
triple = lambda x : x*3

# Q1:Product

Write a function called product that returns the product of the first n terms of a sequence. Specifically, product takes in an integer n and term, a single-argument function that determines a sequence. (That is, term(i) gives the ith term of the sequence.) product(n, term) should return term(1) * ... * term(n).

In [4]:
def product(n, term):
    """Return the product of the first n terms in a sequence.

    n: a positive integer
    term:  a function that takes one argument to produce the term

    >>> product(3, identity)  # 1 * 2 * 3
    6
    >>> product(5, identity)  # 1 * 2 * 3 * 4 * 5
    120
    >>> product(3, square)    # 1^2 * 2^2 * 3^2
    36
    >>> product(5, square)    # 1^2 * 2^2 * 3^2 * 4^2 * 5^2
    14400
    >>> product(3, increment) # (1+1) * (2+1) * (3+1)
    24
    >>> product(3, triple)    # 1*3 * 2*3 * 3*3
    162
    """
    "*** YOUR CODE HERE ***"
    if n == 1:
        return term(1)
    else:
        return term(n)*product(n-1,term)

print(product(3,identity))
print(product(5,identity))
print(product(3,square))
print(product(5,square))
print(product(3,increment))
print(product(3,triple))

6
120
36
14400
24
162


# Q2:Accumulate

Let's take a look at how product is an instance of a more general function called accumulate, which we would like to implement:

In [15]:
def accumulate(fuse, start, n, term):
    """Return the result of fusing together the first n terms in a sequence
    and start.  The terms to be fused are term(1), term(2), ..., term(n).
    The function fuse is a two-argument commutative & associative function.

    >>> accumulate(add, 0, 5, identity)  # 0 + 1 + 2 + 3 + 4 + 5
    15
    >>> accumulate(add, 11, 5, identity) # 11 + 1 + 2 + 3 + 4 + 5
    26
    >>> accumulate(add, 11, 0, identity) # 11 (fuse is never used)
    11
    >>> accumulate(add, 11, 3, square)   # 11 + 1^2 + 2^2 + 3^2
    25
    >>> accumulate(mul, 2, 3, square)    # 2 * 1^2 * 2^2 * 3^2
    72
    >>> # 2 + (1^2 + 1) + (2^2 + 1) + (3^2 + 1)
    >>> accumulate(lambda x, y: x + y + 1, 2, 3, square)
    19
    """
    "*** YOUR CODE HERE ***"
    # 这里是用递归实现的，在n较大的时候可能会导致栈溢出
    # if n == 0:
    #     return start
    # elif n == 1:
    #     return fuse(start,term(1))
    # else:
    #     return fuse(term(n),accumulate(fuse,start,n-1,term))
    # 这里是用迭代实现
    total = start
    for i in range(1,n+1):
        total = fuse(total,term(i))
    return total
print(accumulate(add, 0, 5, identity))
print(accumulate(add, 11, 5, identity))
print(accumulate(add, 11, 0, identity))
print(accumulate(add, 11, 3, square))
print(accumulate(mul, 2, 3, square))
print(accumulate(lambda x, y: x + y + 1, 2, 3, square))

15
26
11
25
72
19


accumulate has the following parameters:
- fuse: a two-argument function that specifies how the current term is fused with the previously accumulated terms
- start: value at which to start the accumulation
- n: a non-negative integer indicating the number of terms to fuse
- term: a single-argument function; term(i) is the ith term of the sequence

Implement accumulate, which fuses the first n terms of the sequence defined by term with the start value using the fuse function.

Assume that fuse is commutative, fuse(a, b) == fuse(b, a), and associative, fuse(fuse(a, b), c) == fuse(a, fuse(b, c)).

Then, implement summation (from lecture) and product as one-line calls to accumulate.

Important: Both summation_using_accumulate and product_using_accumulate should be implemented with a single line of code starting with return.

【注】：这里应用了函数柯里化

In [16]:
def summation_using_accumulate(n, term):
    """Returns the sum: term(1) + ... + term(n), using accumulate.

    >>> summation_using_accumulate(5, square) # square(1) + square(2) + ... + square(4) + square(5)
    55
    >>> summation_using_accumulate(5, triple) # triple(1) + triple(2) + ... + triple(4) + triple(5)
    45
    >>> # This test checks that the body of the function is just a return statement.
    >>> import inspect, ast
    >>> [type(x).__name__ for x in ast.parse(inspect.getsource(summation_using_accumulate)).body[0].body]
    ['Expr', 'Return']
    """
    return accumulate(add,0,n,term)

print(summation_using_accumulate(5, square))
print(summation_using_accumulate(5, triple))

55
45


In [17]:
def product_using_accumulate(n, term):
    """Returns the product: term(1) * ... * term(n), using accumulate.

    >>> product_using_accumulate(4, square) # square(1) * square(2) * square(3) * square()
    576
    >>> product_using_accumulate(6, triple) # triple(1) * triple(2) * ... * triple(5) * triple(6)
    524880
    >>> # This test checks that the body of the function is just a return statement.
    >>> import inspect, ast
    >>> [type(x).__name__ for x in ast.parse(inspect.getsource(product_using_accumulate)).body[0].body]
    ['Expr', 'Return']
    """
    return accumulate(mul,1,n,term)
print(product_using_accumulate(4, square))
print(product_using_accumulate(6, triple))

576
524880


# Q3:Make Repeater

Implement the function make_repeater which takes a one-argument function f and a positive integer n. It returns a one-argument function, where make_repeater(f, n)(x) returns the value of f(f(...f(x)...)) in which f is applied n times to x. For example, make_repeater(square, 3)(5) squares 5 three times and returns 390625, just like square(square(square(5))).

In [21]:
def make_repeater(f, n):
    """Returns the function that computes the nth application of f.

    >>> add_three = make_repeater(increment, 3)
    >>> add_three(5)
    8
    >>> make_repeater(triple, 5)(1) # 3 * (3 * (3 * (3 * (3 * 1))))
    243
    >>> make_repeater(square, 2)(5) # square(square(5))
    625
    >>> make_repeater(square, 3)(5) # square(square(square(5)))
    390625
    """
    "*** YOUR CODE HERE ***"
    # 方法一：使用递归
    # if n == 0:
    #     return lambda x:x # 0次应用，直接返回原值
    # elif n == 1:
    #     return f # 1次应用，直接返回f
    # else:
    #     return lambda x:f(make_repeater(f,n-1)(x))
    #-----------------------------------------------#
    # 方法二：使用迭代
    def repeater(x):
        result = x
        for _ in range(n):
            result = f(result)
        return result
    return repeater

add_three = make_repeater(increment, 3)
add_three(5)

8